# Imports

In [ ]:

!pip install transformers
from transformers import DistilBertTokenizerFast 
import numpy as np
import pandas as pd
import pickle


# Intent = greetings

In [ ]:

from tqdm.auto import tqdm

# below are some common names, surnames and other identities stored by an account holder to transfer money
first_names = ['priya' , 'tanya' , 'priyanka' , 'divya' , 'tanvi' , 'ishita' , 'vani' , 'anjali' , 'shreya' , 'riya' , 'rohan' , 'mayur' ,
               'rahul' , 'abhishek' , 'aditya' , 'amit' , 'mahesh' , 'rohit' , 'yash' , 'ankit' , 'shyam' , 'deepak' , 'vivek' ]
last_names = ['thakur' , 'singh' , 'reddy' , 'das' , 'kaur' , 'anand' , 'yadav' , 'lal' , 'patel' , 'shinde' , '' , '' , '' ]
other_identities = [ 'bank' , 'milk' , 'grocery' , 'vegetables' , 'business' , 'hospital' , 'food' , 'school' , 'college' , 'service' , 
                    'education' , 'medicines' , 'repair' , 'electricity'] 

# randomly generating identity from above lists
def identity_generator( n , name_share = 0.5 ):

    identity = []
    for i in range(int(name_share*n)):
        te = np.random.choice(last_names , 1)[0]
        if te == '':
            identity.append(np.random.choice(first_names , 1)[0])
        else:
            identity.append(np.random.choice(first_names , 1)[0] + ' ' + te)

    for i in range(int((1-name_share)*n)):
        identity.append(np.random.choice(other_identities , 1)[0])
    
    return identity



def expand_questions( sentences , named_entities , ner):
    
    '''works for entities of 1 or 2 word length, edit marked code for more types of enitities'''

    iters = 1
    for i in named_entities.values():
        iters *= len(i)

    updated_sentences = []
    tags = []
    for i in sentences:
       
        check = 0
        for j in named_entities.keys():
            if j in i:
                check += 1

        if check == 0:

            te1 = []
            te2 = []
            for j in i.split(' '):
                te1.append(j)
                te2.append('o')

            updated_sentences.append(te1)
            tags.append(te2)
        
        else:

            for iteration in range(iters):

                te1 = []
                te2 = []
                for j in i.split(' '):

                    if j[0] == '#':
                        ent = np.random.choice( named_entities[j] )

                        # Edit this part to support more word length
                        for l,k in enumerate(ent.split(' ')):
                            te1.append(k)
                            te2.append( ner.get( j , ['o']*10 )[l] )
                        # Uptill here 
                    
                    else:
                        te1.append(j)
                        te2.append('o')
                        
                updated_sentences.append(te1)
                tags.append(te2)
    
    return updated_sentences , tags


#################################################################################################################################


greetings = [
             
'good morning' , 'good evening' , 
'hi' , 'hey' , 'hello' , 'hi there' , 'hey there' , 'hello there' ,
'hello are you still there' , 'hello its #identity' # the '#identity' in this string will get expanded by function - 'expand_questions'

] 



In [ ]:

identity = identity_generator(50 , name_share = 1)
named_entities = {'#identity' : identity}
ner = {'#identity' : ('s-id','e-id')}
greetings , greetings_tags = expand_questions( greetings , named_entities , ner)
greetings_intent = ['greetings']*len(greetings)


In [ ]:
greetings[40:45]

[['hello', 'its', 'deepak', 'thakur'],
 ['hello', 'its', 'anjali', 'anand'],
 ['hello', 'its', 'abhishek', 'shinde'],
 ['hello', 'its', 'tanvi', 'shinde'],
 ['hello', 'its', 'anjali', 'patel']]

In [ ]:

# link about entity marking conventions
# https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)
greetings_tags[40:45]


[['o', 'o', 's-id', 'e-id'],
 ['o', 'o', 's-id', 'e-id'],
 ['o', 'o', 's-id', 'e-id'],
 ['o', 'o', 's-id', 'e-id'],
 ['o', 'o', 's-id', 'e-id']]

# Intent = bank_related

In [ ]:

debt_instruments = [ 'vehicle loan' , 'business loan' , 'personal loan' , 'loan' , 'education loan' , 'medical loan' , 'mortgage']
credit_cards = ['silver card' , 'gold card' , 'platinum card' , 'credit card' ]

bank_related = [

# Open a new account 
'open an account at fake bank',
'open a new account with chatbot',
'sign up for an account',
'sign up',
'open account',

# Log in
'log into my account',
'sign into my account',
'login',
'sign in',

# Add beneficiary 
'add #identity as benefeciary',
'add #identity to benefeciary list',
'add #identity to account',

# Edit beneficiary 
'edit beneficiary name for #identity',
'edit #identity name',
'change beneficiary name from #identity',
'change this name from #identity to',

# Remove beneficiary 
'remove #identity from beneficiaries list',
'remove #identity',
'remove a beneficiary #identity from beneficiaries',       

# Apply for bank products
'get a new #debt_instruments',
'get a new #credit_cards',
'apply for #credit_cards',
'apply for #debt_instruments',

# Discontinue credit cards
'discontinue #credit_cards services',
'remove #credit_cards services',
'discontinue #credit_cards',

# Close account
'delete my account',
'close my account',

# Forgot account details
'forgot username',
'forgot account no',
'forgot password what to do',
'i do not remember my account no',
'i do not remember my password',

]



In [ ]:

identity = identity_generator(50 , name_share = 0.6)
named_entities = {'#identity' : identity , '#debt_instruments' : debt_instruments , '#credit_cards' : credit_cards}
ner = {'#identity' : ('s-id','e-id') , '#debt_instruments' : ('s-product','e-product') , '#credit_cards' : ('s-product','e-product')}
bank_related , bank_related_tags = expand_questions( bank_related , named_entities , ner)
bank_related_intent = ['bank_related']*len(bank_related)


# Intent = transactions

In [ ]:

emi = ['emi' , 'monthly installments' , 'installments this year' , 'interest' ]

transactions = [
               
# Pay beneficiary / friend / other identities
'pay #identity 2500',
'send 85445 to #identity',
'give #identity 589',
'please send 55785 to #identity',
'pay #identity 5000 using #credit_cards',
'send 7999 to #identity using #credit_cards',
'give #identity 89900 using #credit_cards',
'please send 800 to #identity using #credit_cards',

# Pay bank
'pay #emi due',
'pay #debt_instruments due',
'pay #credit_cards due',
'pay my #emi',
'pay my #debt_instruments bill',
'pay my #credit_cards bill',

# Use credit card to pay other debts
'pay #emi due using #credit_cards',
'pay #debt_instruments due using #credit_cards',

]



In [ ]:

identity = identity_generator(30 , name_share = 0.6)
named_entities = {'#identity' : identity , '#debt_instruments' : debt_instruments , '#credit_cards' : credit_cards , '#emi' : emi}
ner = {'#identity' : ('s-id','e-id') , '#debt_instruments' : ('s-product','e-product') , '#credit_cards' : ('s-product','e-product') }
transactions , transactions_tags = expand_questions( transactions , named_entities , ner)
transactions_intent =  ['transactions_intent']*len(transactions)


# Intent = analyze_transactions

In [ ]:

bills = [ 'bank fees' , 'credit card bills', 'electricity bills' , 'hospital bills' , 'education fees' , 'medicine bills' , 'repair charges'] 
time_period = ['in past 2 months' , 'in past 6 days' , 'in past week' ,  'yesterday' , 'today' ]


analyze_transactions = [
                       
# Deposit type transactions
'when was the last time money was deposited',
'how much money was credited to my account #time_period',
'how much money amount went into my account #time_period',
'how much money amount put into my account #time_period',
'how much money amount #identity sent me #time_period',
'how much money amount #identity paid me #time_period',
'how much was deposited to my account #time_period',
'money amount received from #identity #time_period',
'show total amount of deposits #time_period',
'check total amount of deposits #time_period',

# Withdrawal type transactions
'when was the last time money was withdrawn'
'how much money was debited from account #time_period',
'how much money amount went out of my account #time_period',
'how much money amount was paid to #identity #time_period',
'how much money amount was sent to #identity #time_period',
'#bills paid #time_period',
'show total money amount of withdrawals #time_period',
'show total money amount of deductions #time_period',
'show total expense #time_period',
'show total spendings #time_period',
'check total money amount of withdrawals #time_period',
'check total money amount of deductions #time_period',
'check total spendings #time_period',
'how much money was withdrawn from my account #time_period',
'how much money was taken out of my account #time_period',
'how much money was cut from my account #time_period',

# Sent and received breakdown 
'how much money was transacted from my account #time_period',
'transactions between me and #identity #time_period',
'get transactions breakdown #time_period',
'show total money amount of transactions #time_period',
'check total money amount of transactions #time_period',

# Credit card breakdowns
'when did i use my #credit_cards last time',
'show total spendings using #credit_cards #time_period',
'how much cashbacks i received using #credit_cards #time_period',

]


In [ ]:

identity = identity_generator(50 , name_share = 0.35)
named_entities = {'#identity' : identity , '#credit_cards' : credit_cards  , '#bills' : bills , '#time_period' : time_period}
ner = {'#identity' : ('s-id','e-id') , '#credit_cards' : ('s-product','e-product') }
analyze_transactions , analyze_transactions_tags = expand_questions( analyze_transactions , named_entities , ner)
analyze_transactions_intent =  ['analyze_transactions']*len(analyze_transactions)


# Intent = balances

In [ ]:


balances = [

# Check account balance            
'what is my account balance',
'can you tell me how much money is left in my account',
'what is the left in my account',
'check balance for account',
'show account balance',
'tell me my account balance',

# Check credit card balance 
'how much more can i spend using my #credit_cards',
'credit balance remaining in #credit_cards',
'what limit of my #credit_cards is exhausted',
'what is my #credit_cards limit', #tips and tricks

# Check debt balances (3 types of questions clubbed together)
'check #debt_instruments remaining amount',
'show principal remaining on #debt_instruments',
'principal amount left on my #debt_instruments',
'principal yet to be returned on my #debt_instruments',
'total amount yet to be paid for my #debt_instruments',

'check #debt_instruments repaid amount',
'show principal returned from #debt_instruments',
'principal amount paid on my #debt_instruments till now',

'what was my initial #debt_instruments amount',
'what was my initial principal amount',
'how much was my initial #debt_instruments amount',
'what was my original principal amount',

]



In [ ]:

named_entities = {'#debt_instruments' : debt_instruments , '#credit_cards' : credit_cards}
ner = {'#debt_instruments' : ('s-product','e-product') , '#credit_cards' : ('s-product','e-product') }
balances , balances_tags = expand_questions( balances , named_entities , ner)
balances_intent =  ['balances']*len(balances)


# Intent = check_loan_details

In [ ]:

check_loan_details = [

# Loan emi details
'#debt_instruments #emi',
'how much #debt_instruments #emi i need to pay',
'#emi due i need to pay this month for #debt_instruments',
'what is the #emi due for this month',
'show monthly #emi for #debt_instruments',
'what is my #debt_instruments dues',

# Due date for loan
'what is the due date of #emi payment for #debt_instruments',
'what is the last date of #emi payment for #debt_instruments',
'no of remaining days for #emi payment for #debt_instruments',
'last date of #debt_instruments payment',
'next due date of #emi payment for #debt_instruments',
'next repayment date of #emi payment for #debt_instruments',
'when will my #emi for #debt_instruments get deducted',

# Maturity date
'time left on my #debt_instruments',
'how many years left for #debt_instruments', 
'months left on #debt_instruments', 
'duration of my #debt_instruments',
'my #debt_instruments tenure',
'when is my #debt_instruments ending',

# General loan related questions
'what are the different type of loans available', 
'what can i use a #debt_instruments for',
'how is the maximum #debt_instruments amount decided',
'what is the tenure of a #debt_instruments',
'what are the documents required when applying for a #debt_instruments',
'what are the interest rates for #debt_instruments',
'how much interest is charged for #debt_instruments',
'what are possible collaterals for #debt_instruments',
'what will be kept in banks possesion in exchange for #debt_instruments',

]


In [ ]:

named_entities = {'#debt_instruments' : debt_instruments , '#emi' : emi}
ner = {'#debt_instruments' : ('s-product','e-product') }
check_loan_details , check_loan_details_tags = expand_questions( check_loan_details , named_entities , ner )
check_loan_details_intent =  ['check_loan_details']*len(check_loan_details)


# Intent = check_credit_card_details

In [ ]:

check_credit_card_details = [

# Credit card bill
'what is my #credit_cards bill this month',
'show #credit_cards bill this month',
'display #credit_cards bill this month',
'what are my #credit_cards dues', 
'what is my #credit_cards installment this month',
'show #credit_cards installment this month',
'#credit_cards installment',

# Offers
'#credit_cards offers available for #identity',
'#credit_cards any discounts for #identity',
'#credit_cards promos available for #identity',
'any #credit_cards promotions for #identity',
'what type of benefits can #credit_cards provide',

# Limit related questions                     
'what is my maximum limit for #credit_cards',
'#credit_cards allowed expense on credit',
'is my #credit_cards over limit',
'has my #credit_cards limit reached',

# Interest rate
'what is my #credit_cards interest rate',
'what is interest rate for #credit_cards',
'show #credit_cards interest rate',

# General questions 
'what is a credit card',
'please explain about credit card',
'how does a credit card differ from a debit card',
'how much will be my next bill if i dont pay this month',
'what are the different types of credit card',
'what is an annual fee for #credit_cards',
'what are the charges of #credit_cards',
'when will be emi installments money collected this month for #credit_cards',
'how much interest rate is charged for #credit_cards',
'what is payment due date for #credit_cards bills',
'next repayment date for #credit_cards bills',

]


In [ ]:

identity = identity_generator(50 , name_share = 0.35)
named_entities = {'#identity' : identity , '#credit_cards' : credit_cards}
ner = {'#identity' : ('s-id','e-id') , '#credit_cards' : ('s-product','e-product') }
check_credit_card_details , check_credit_card_details_tags = expand_questions( check_credit_card_details , named_entities , ner)
check_credit_card_details_intent  =  ['check_credit_card_details']*len(check_credit_card_details)


# Intent = chatbot_related

In [ ]:

chatbot_related = [

# Chatbot capabilities 
'what could be done with chatbot',     
'what services can chatbot offer', 
'what is the scope of chatbot',
'what can chatbot do',
'what services can i use',
'how to use the chatbot',
'what are the features of the chatbot',


# Chatbot tracking expenses
'how can i keep track of money sent and received',
'is it possible analyze spendings',
'how can i keep track of transactions',
'is it possible to keep track of money sent and received',
'how can i analyze spendings',

]


In [ ]:

identity = identity_generator(50 , name_share = 0.35)
named_entities = {}
ner = {}
chatbot_related , chatbot_related_tags = expand_questions( chatbot_related , named_entities , ner)
chatbot_related_intent =  ['chatbot_related']*len(chatbot_related)


# Final dataset

In [ ]:

sentences = greetings + bank_related + transactions + analyze_transactions + balances + check_loan_details + check_credit_card_details + chatbot_related

labels = greetings_tags + bank_related_tags + transactions_tags + analyze_transactions_tags + balances_tags + \
         check_loan_details_tags + check_credit_card_details_tags + chatbot_related_tags

intents = greetings_intent + bank_related_intent + transactions_intent + analyze_transactions_intent + \
          balances_intent + check_loan_details_intent + check_credit_card_details_intent + chatbot_related_intent


In [ ]:

temp = pd.DataFrame( {'sentences' : sentences , 'labels' : labels , 'intents' : intents} )
temp


,sentences,labels,intents
0,"[good, morning]","[o, o]",greetings
1,"[good, evening]","[o, o]",greetings
2,[hi],[o],greetings
3,[hey],[o],greetings
4,[hello],[o],greetings
...,...,...,...
303225,"[how, can, i, keep, track, of, money, sent, an...","[o, o, o, o, o, o, o, o, o, o]",chatbot_related
303226,"[is, it, possible, analyze, spendings]","[o, o, o, o, o]",chatbot_related
303227,"[how, can, i, keep, track, of, transactions]","[o, o, o, o, o, o, o]",chatbot_related
303228,"[is, it, possible, to, keep, track, of, money,...","[o, o, o, o, o, o, o, o, o, o, o]",chatbot_related


In [ ]:

temp['labels'] = temp['labels'].apply( lambda x : ','.join(x) )
temp['sentences'] = temp['sentences'].apply( lambda x : ','.join(x) )
temp.drop_duplicates( inplace = True , ignore_index = True )
temp['labels'] = temp['labels'].apply( lambda x : x.split(',') )
temp['sentences'] = temp['sentences'].apply( lambda x : x.split(',') )
temp


,sentences,labels,intents
0,"[good, morning]","[o, o]",greetings
1,"[good, evening]","[o, o]",greetings
2,[hi],[o],greetings
3,[hey],[o],greetings
4,[hello],[o],greetings
...,...,...,...
2981,"[how, can, i, keep, track, of, money, sent, an...","[o, o, o, o, o, o, o, o, o, o]",chatbot_related
2982,"[is, it, possible, analyze, spendings]","[o, o, o, o, o]",chatbot_related
2983,"[how, can, i, keep, track, of, transactions]","[o, o, o, o, o, o, o]",chatbot_related
2984,"[is, it, possible, to, keep, track, of, money,...","[o, o, o, o, o, o, o, o, o, o, o]",chatbot_related


In [ ]:

with open('chatbot_data' , 'wb') as f:
    pickle.dump((temp) , f)
